### URL parsing

In [6]:
import urllib.parse

# 한글 문자열
text = "강남역 맛집"

# URL 인코딩
encoded_text = urllib.parse.quote(text)

base_path=f"https://www.google.co.kr/maps/search/{encoded_text}"

print(encoded_text)

%EA%B0%95%EB%82%A8%EC%97%AD%20%EB%A7%9B%EC%A7%91


### googlemap_info

In [83]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.select import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located,presence_of_element_located

Main_Drive=Chrome()
Main_Drive.get(base_path)

main_box=WebDriverWait(Main_Drive,3).until(
    presence_of_element_located([By.XPATH, '//*[@role="feed"]']))

while True:
    main_box.send_keys(Keys.PAGE_DOWN)

    feed_box=WebDriverWait(Main_Drive,3).until(
        visibility_of_all_elements_located([By.CLASS_NAME,"hfpxzc"]))
    
    if len(feed_box)>=20:
        break

### Restaurant_info

In [90]:
{
    "검색어" : {"rich_text":[{"text": {"content": text}}]},
    
}

{'검색어': {'rich_text': [{'text': {'content': 'test_info'}}]}}

In [ ]:
{'URL': {'id': '%3EJu%3A', 'type': 'url', 'url': None},
 '검섹어': {'id': 'F%3A%40s', 'type': 'rich_text', 'rich_text': []},
 '별점': {'id': '%5CtsJ', 'type': 'number', 'number': None},
 '주소': {'id': 'cNJq', 'type': 'rich_text', 'rich_text': []},
 '카테고리': {'id': 'lVsU', 'type': 'rich_text', 'rich_text': []},
 'Created': {'id': 'mxHN',
  'type': 'created_time',
  'created_time': '2024-12-18T09:48:00.000Z'},
 'DB': {'id': 'vQV%7D', 'type': 'relation', 'relation': [], 'has_more': False},
 '음식점이름': {'id': 'title',
  'type': 'title',
  'title': [{'type': 'text',
    'text': {'content': '이건뭐야?', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': '이건뭐야?',
    'href': None}]}}

In [88]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [78]:
main_box.get_attribute("role")

'feed'

In [72]:
feed_box[0].get_attribute("aria-label")

'바비레드 강남본점'

In [89]:
Main_Drive.quit()

In [59]:
test_elements=Main_Drive.find_elements(By.CLASS_NAME,"hfpxzc")

MaxRetryError: HTTPConnectionPool(host='localhost', port=57278): Max retries exceeded with url: /session/099672721e87bc05898169d8f5370121/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109ffabb0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [47]:
test_elements[-1].get_attribute("aria-label")

'땀땀'

In [48]:
test_elements[-1].get_attribute("href")

'https://www.google.co.kr/maps/place/%EB%95%80%EB%95%80/data=!4m7!3m6!1s0x357cbba6cab8be71:0x123ebf3924dc0759!8m2!3d37.5003867!4d127.027994!16s%2Fg%2F11hcqrdf1p!19sChIJcb64yqa7fDURWQfcJDm_PhI?authuser=0&hl=ko&rclk=1'

In [49]:
Main_Drive.quit()

In [22]:
# 검색창
feedbox=Main_Drive.find_elements(By.TAG_NAME,"div")

In [25]:
feed_test=[element for element in feedbox if element.get_attribute("role")=="feed"]

In [29]:
feed_test[0].get_attribute("aria-label")

'강남역 맛집에 대한 검색결과'

In [33]:
from selenium.webdriver.common.keys import Keys

for _ in range(10):
    feed_test[0].send_keys(Keys.PAGE_DOWN)

테스트 끝.

In [37]:
Main_Drive.quit()

In [40]:
from threading import Thread

구조 

step 1 : url get

step 2 : url을 통해 세부정보 get